In [1]:
print(__doc__)

from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

Automatically created module for IPython interactive environment


In [2]:
np.random.seed(42)

iris = datasets.load_iris()
print iris.data.shape
print iris.target.shape

data = scale(iris.data)

n_samples, n_features = data.shape
n_digits = len(np.unique(iris.target))
print iris.target

print np.unique(iris.target)
labels = iris.target

sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))

(150L, 4L)
(150L,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[0 1 2]
n_digits: 3, 	 n_samples 150, 	 n_features 4


In [3]:
from sklearn import svm
clf=svm.LinearSVC()
# learn from existing data using estimator called fit(x,y)
clf.fit(iris.data,iris.target)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [4]:
# use model to predict most likely outcome on unseen data

clf.predict([[5.0,3.6,1.3,0.25]])
clf.coef_ #access parameters of model via attributes

array([[ 0.18423281,  0.45122677, -0.80794335, -0.45071701],
       [ 0.05182102, -0.89410543,  0.40512989, -0.93782962],
       [-0.85076016, -0.98680704,  1.38094499,  1.86551338]])

In [5]:
# classification:

from sklearn import neighbors

knn = neighbors.KNeighborsClassifier()
knn.fit(iris.data,iris.target)

knn.predict([[0.1,0.2,0.3,0.4]])

array([0])

In [6]:
perm = np.random.permutation(iris.target.size)
iris.data = iris.data[perm]
iris.target = iris.target[perm]
knn.fit(iris.data[:100],iris.target[:100])

knn.score(iris.data[100:],iris.target[100:])

0.95999999999999996

In [7]:
#K-mean clustering:

from sklearn import cluster,datasets
iris = datasets.load_iris()
k_means = cluster.KMeans(3) #three parameters
k_means.fit(iris.data)

print k_means.labels_[::10]
print iris.target[::10]

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


In [8]:
print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')


def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(79 * '_')

_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
k-means++   0.02s    141   0.641   0.644   0.643   0.592   0.637    0.462
   random   0.03s    141   0.641   0.644   0.643   0.592   0.637    0.462
PCA-based   0.00s    142   0.660   0.662   0.661   0.645   0.656    0.455
_______________________________________________________________________________


In [9]:
import numpy as np
from sklearn import cross_validation, datasets, decomposition, svm
import glob
%matplotlib inline
# ..
# .. load data ..


spiders = glob.glob('./Spider/*')
# print spiders
fnlist = []
fnlist[0:len(spiders)-1] = spiders[:]
print np.shape(fnlist)
# print spiders[:]

img = []
for i in range(0,len(spiders)):
    img.append(mpimg.imread(spiders[i]))
    

from sklearn import cluster,datasets
n_samples = len(img)
for i in img:
    data = i.reshape((n_samples, -1))

lfw_people = datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)
faces = np.reshape(lfw_people.data, (lfw_people.target.shape[0], -1))
train, test = iter(cross_validation.StratifiedKFold(lfw_people.target, k=4)).next()
X_train, X_test = faces[train], faces[test]
y_train, y_test = lfw_people.target[train], lfw_people.target[test]

# ..
# .. dimension reduction ..
# pca = decomposition.RandomizedPCA(n_components=150, whiten=True)
# pca.fit(X_train)
# X_train_pca = pca.transform(X_train)
# X_test_pca = pca.transform(X_test)

# # ..
# # .. classification ..
# clf = svm.SVC(C=5., gamma=0.001)
# clf.fit(X_train_pca, y_train)

# print 'Score on unseen data: '
# print clf.score(X_test_pca, y_test)

(565L,)


ValueError: total size of new array must be unchanged

In [10]:
import  sklearn as sk
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy
from PIL import Image
# from sklearn.neural_network import MLPClassifier

spiders = glob.glob('./Spider/*')
non_spiders = glob.glob('./NonSpider/*')

SPimg = []
for sp in spiders:
    img = Image.open(sp)
    resized = scipy.misc.imresize(img,[400,400])
    SPimg.append(resized)

NSPimg = []
for sp in non_spiders:
    img = Image.open(sp)
    resized = scipy.misc.imresize(img,[400,400])
    NSPimg.append(resized)

testSize = int(0.75*len(SPimg)) + int(0.75*len(NSPimg))
checkSpid = int(0.25*len(SPimg))
checkNSpid = int(0.25*len(NSPimg))
testLabs = np.zeros(testSize)

testImgs = SPimg[0:int(0.75*len(SPimg))]

for i in range(0,int(0.75*len(SPimg))):
    testLabs[i] = 1
# testLabs = np.ones(int(0.75*len(SPimg)))

testImgs[len(testImgs):(len(testImgs)+(int)(0.75*len(NSPimg)))] = SPimg[0:(int)(0.75*len(NSPimg))]
toAloc = -1*np.ones(int(0.75*len(NSPimg)))

for i in range(int(0.75*len(NSPimg)),(len(testImgs)-2)):
    testLabs[i] = -1

testImgs = np.array(testImgs)
TwoDim_dataset = testImgs.reshape(testSize,-1)

#

knn = neighbors.KNeighborsClassifier()
knn.fit(TwoDim_dataset,testLabs)


# print testSP
# blah=[]

# for i in testSP:
#     blah1=[]
#     for j in i:
#         blah1.append(j)
#     blah.append(blah1)
# print testSP
# print blah

# blah = np.array(blah)
print len(SPimg)
# testSP = SPimg[int(0.75*len(SPimg)):230]
testSP = NSPimg[int(0.75*len(NSPimg)):len(NSPimg)]
print 0.75*len(SPimg)
check = np.array(testSP)

TwoDtestSP=check.reshape(len(NSPimg)-int(0.75*len(NSPimg)),-1)

knn.predict(TwoDtestSP)

# knn.score(TwoDtestSP)




# np.squeeze(x, axis=(2,)).shape
# (1, 3)
# check=np.array(testSP)
# check=[item for sublist in x for item in sublist]
# # TwoDtestSP = check.reshape(checkSpid,-1)

# knn.predict(check)
#Now for a single layer perceptron with 3 nodes in the hidden layer
# SLPClass = MLPClassifier(hidden_layer_sizes=(3,))
# SLPClass.fit(TwoDim_dataset,testLabs)

# print SLPClass.loss_
# testSP = np.array(SPimg[(int)(0.75*len(SPimg))+1:-1])
# testSP = testSP.reshape(checkSpid,-1)
# testSLPSPScore = SLPClass.score(testSP,np.ones(len(testSP)))
# print "Testing the SLP on spiders " , testSLPSPScore
#
# testNSP = NSPimg[(int)(0.75*len(NSPimg))+1:-1]
#
# # testNSLPSPScore = SLPClass.score(testNSP,np.ones(len(testNSP)))
# print "Testing the SLP on non-spiders " , testNSLPSPScore

ValueError: could not broadcast input array from shape (400,400,3) into shape (400,400)

In [ ]:
knn.predict_proba(TwoDtestSP)


In [ ]:

plt.imshow(SPimg[int(213.5)+2])